# This code predicts weather a person is likely to have PD or not based on his features

### Importing libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logit

ImportError: No module named logit

In [15]:
df=pd.read_csv("Beforemedicationbaselineupdrs3.csv")

In [16]:
df.head()

,Unnamed: 0,PATNO,PAG_NAME,NP3FACXP,NP3SPCH,NP3RIGN,NP3RIGRU,NP3RIGLU,PN3RIGRL,NP3RIGLL,...,NP3RTALU,NP3RTARL,NP3RTALL,NP3RTALJ,NP3RTCON,DYSKPRES,DYSKIRAT,NHY,PD_MED_USE,DIAG
0,0,3816,NUPDRS3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,0,0,NotPD
1,1,3762,NUPDRS3,1,0,0,1,1,0,0,...,0,0,0,0,1,1,1,2,0,PD
2,2,3765,NUPDRS3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,0,0,NotPD
3,3,3075,NUPDRS3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,0,0,NotPD
4,4,3760,NUPDRS3,1,1,1,0,1,0,0,...,0,0,0,0,0,0,NaN,2,0,PD


In [9]:
df.tail()

,NP3SPCH,NP3FACXP,NP3RIGN,NP3RIGRU,NP3RIGLU,PN3RIGRL,NP3RIGLL,NP3FTAPR,NP3FTAPL,NP3HMOVR,...,NP3RTARU,NP3RTALU,NP3RTARL,NP3RTALL,NP3RTALJ,NP3RTCON,DYSKPRES,NHY,PD_MED_USE,DIAG
1251,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NotPD
1252,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,NotPD
1253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NotPD
1254,1,1,2,3,3,1,2,1,1,0,...,0,1,0,0,0,1,0,2,7,PD
1255,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NotPD


### Deleting unnecessary columns 

In [17]:
del df["DYSKIRAT"]
del df["PATNO"]
del df["Unnamed: 0"]
del df["PAG_NAME"]
#del df["DIAG"]
del df["PD_MED_USE"]

In [21]:
df.isnull().values.any() #finding if there are any null values

False

In [22]:
len(df)

1250

In [20]:
df=df.dropna()

## Finding if there are any correlations

In [10]:
def plot_corr(df,size=11):
    corr=df.corr()
    fig,ax=plt.subplots(figsize=(size,size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)),corr.columns)
    plt.yticks(range(len(corr.columns)),corr.columns)
    

In [47]:
#%matplotlib inline
plot_corr(df)

In [48]:
plt.show()

In [16]:
test=df.corr()

In [14]:
def TT(test,column):
    a=test[test[str(column)]>0.5]
    return a[str(column)]

In [75]:
from IPython.display import clear_output
for i in test.columns:
    print i
    print TT(test,i)
    a=raw_input("Press Enter to continue...")
    clear_output()


In [90]:
#del df["NP3LGAGL"]
#del df["NP3LGAGR"]
del df["DYSKPRES"]

In [91]:
del df["NP3RIGRU"]

In [92]:
del df["NP3RIGLU"]
del df["NP3FTAPL"]
del df["NP3FTAPR"]
del df["NP3PRSPR"]
del df["NP3PRSPL"]

In [93]:
del df["NP3TTAPL"]
del df["NP3TTAPR"]

In [94]:
del df["NP3FACXP"]
del df["NHY"]

### CHECK TRUE OR FALSE RATIO


In [9]:
#diabetes_map={True:1,False:0}
#df["diabetes"]=df["diabetes"].map(diabetes_map)

In [10]:
num_true=len(df.loc[df["DIAG"] == "PD"])
num_false=len(df.loc[df["DIAG"] == "NotPD"])
print("Number of True cases: ", (num_true,(num_true/(num_true+num_false))*100))
print("Number of False cases: ", (num_false,(num_false/(num_true+num_false))*100))


('Number of True cases: ', (672, 0))
('Number of False cases: ', (578, 0))


### Splitting the data 

In [23]:
from sklearn.cross_validation import train_test_split


In [24]:
features=df.columns.delete(-1)
predicted=df.columns-features
X=df[features].values
Y=df[predicted].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [10]:
#Y_test

### Checking Zeroes

In [39]:
print "rows in dataframe ",len(df)
print "rows missing glucose conc ", len(df.loc[df["glucose_conc"] == 0])
print "rows misssing bp", len(df.loc[df["diastolic_bp"]==0])
print "rows missing insulin ",len(df.loc[df["insulin"]==0])
print "rows missing bmi ",len(df.loc[df["bmi"] == 0])
print "rows missing diabetes predispositon function ",len(df.loc[df["diab_pred"] == 0])
print "rows missing age ",len(df.loc[df["age"] == 0])

rows in dataframe  1252
rows missing glucose conc 

KeyError: 'glucose_conc'

## Imputing Missing values using mean of the column

In [81]:
from sklearn.preprocessing import Imputer

#Imputing missing values with mean
fill_0=Imputer(missing_values=0,strategy="mean",axis=0)
X_train=fill_0.fit_transform(X_train)
X_test=fill_0.fit_transform(X_test)

## Training algorithm logistic regression

In [25]:
from sklearn import linear_model
logit = linear_model.LogisticRegression(C=1e5)
logit.fit(X_train,Y_train.ravel())

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [26]:
print "The coefficients are ",logit.coef_

The coefficients are  [[ 1.14841223 -0.64117228  0.58028057  0.58221171  0.23207188  1.14639573
  -0.39613151  1.3809578  -0.92069318 -1.4114435   0.79250999  0.23372469
   0.56666079 -0.03438597  0.40530408  0.05774863 -1.12404301 -0.8196564
   0.72464922 -0.61875428 -0.5915064   0.16399351  0.7797863  -1.27845943
   0.97193234 -0.80522236 -0.12615661 -0.08260717  0.56236134 -0.49886105
   0.47490169  1.95441319  0.30911931  7.52774436  2.23642196]]


In [46]:
a=abs(logit.coef_[0])
print a

[ 1.14841223  0.64117228  0.58028057  0.58221171  0.23207188  1.14639573
  0.39613151  1.3809578   0.92069318  1.4114435   0.79250999  0.23372469
  0.56666079  0.03438597  0.40530408  0.05774863  1.12404301  0.8196564
  0.72464922  0.61875428  0.5915064   0.16399351  0.7797863   1.27845943
  0.97193234  0.80522236  0.12615661  0.08260717  0.56236134  0.49886105
  0.47490169  1.95441319  0.30911931  7.52774436  2.23642196]


In [47]:
dict={}
for i in range(len(df.columns)-1):
    dict[str(df.columns[i])]=a[i]
    
    

In [48]:
import operator
x = dict
sorted_x = sorted(x.items(), key=operator.itemgetter(1))

In [1]:
sorted_x

NameError: name 'sorted_x' is not defined

In [15]:
print "The Intercept is ",logit.intercept_

The Intercept is  [-4.43038171]


## Predicting the values of test data from the model

In [105]:
def model(x):
    return 1/(1+(np.exp(-x)))

In [106]:
a=(X_test*logit.coef_)
y_pred=[]

In [107]:
for i in a:
    y_pred.append(model(np.sum(i)+logit.intercept_))

In [108]:
y_pred=np.array(y_pred)

In [109]:
#y_pred

## Creating a classification from the predicted value of the model

In [110]:
Y_pred=[]

In [111]:
for i in y_pred:
    if i < 0.5:
        Y_pred.append("NotPD")
    else:
        Y_pred.append("PD")
        

In [112]:
Y_pred=np.array(Y_pred)
Y_pred=Y_pred.ravel()
Y_pred=Y_pred[:,np.newaxis]
#Y_pred

In [113]:
c=Y_test.shape[0]

## Checking the accuracy of the model

In [114]:
fault=0
for i in range(c):
    if Y_test[i] == Y_pred[i]:
    #if Y_test[i] == new_ypred[i]:
        pass
    else:
        fault = fault + 1

In [115]:
print "The accuracy is", (1-(fault/float(c)))*100

The accuracy is 92.5531914894


## Checking weather the Algorithm is intact

In [32]:
new_ypred=[]
for i in X_test:
    new_ypred.append(logit.predict(i.reshape(1,-1)))

In [33]:
new_ypred

[array(['PD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['PD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['NotPD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['PD'], dtype=object),
 array(['P

In [34]:
new_ypred==Y_pred

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

## Prediction of weather a person is likely to have Diabetes based on his features

In [ ]:
df[features].head()


In [ ]:
a=[]
columns=["Glucose_conc","BP","Insulin","BMI","diab_ped","Age"]
no_of_persons=int(raw_input("No of persons"))
for j in range(no_of_persons):
    c=[]
    a.append(c)
    for i in columns:
        d=float(raw_input("Values of "+str(i)))
        c.append(d)

In [ ]:
df1=pd.DataFrame(a,columns=columns)

In [ ]:
X_new=df1.values

In [ ]:
for j in X_new:
    a=logit.predict(j.reshape(1,-1))
    if a==1:
        print "Person has diabetes"
    elif a==0:
        print "Person does not have diabetes"

In [180]:
df.head(5)

,PATNO,PAG_NAME,NP3SPCH,NP3FACXP,NP3RIGN,NP3RIGRU,NP3RIGLU,PN3RIGRL,NP3RIGLL,NP3FTAPR,...,NP3RTALU,NP3RTARL,NP3RTALL,NP3RTALJ,NP3RTCON,DYSKPRES,DYSKIRAT,NHY,PD_MED_USE,DIAG
0,3816,NUPDRS3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,0,0,NotPD
1,3762,NUPDRS3,0,1,0,1,1,0,0,1,...,0,0,0,0,1,1,1,2,0,PD
2,3765,NUPDRS3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,0,0,NotPD
3,3075,NUPDRS3,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,NaN,0,0,NotPD
4,3760,NUPDRS3,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,NaN,2,0,PD


In [181]:
df["PAG_NAME"].value_counts()

NUPDRS3     1256
NUPDRS3A      91
dtype: int64

In [182]:
am=df[df.PAG_NAME=="NUPDRS3A"]

In [187]:
am.to_csv("Aftermedicationbaselineupdrs3.csv")

In [185]:
bf=df[df.PAG_NAME=="NUPDRS3"]

In [188]:
bf.to_csv("Beforemedicationbaselineupdrs3.csv")

In [288]:
def finction(x):
    if str(x) == "PD":
        return int(1)
    else:
        return 0

In [295]:
df["DIAG"]=df["DIAG"].apply(finction)

In [308]:
df.to_csv("combinedadaptedforlogit.csv")

In [44]:
df = pd.DataFrame([[1,2,3],[2,4,5],[3,1,4],[4,2,0]],columns=['A','B','C'])

In [46]:
df

,A,B,C
0,1,2,3
1,2,4,5
2,3,1,4
3,4,2,0


In [29]:
df[df.A> 1] 

,A,B,C
1,2,4,5
2,3,1,4
3,4,2,1


In [48]:
df[(df > 3).any(axis=1)]

,A,B,C
1,2,4,5
2,3,1,4
3,4,2,0


In [55]:
for i in df.columns:
    print df[df[str(i)]>1]

   A  B  C
1  2  4  5
2  3  1  4
3  4  2  0
   A  B  C
0  1  2  3
1  2  4  5
3  4  2  0
   A  B  C
0  1  2  3
1  2  4  5
2  3  1  4


In [8]:
df.std()

NP3SPCH       0.579451
NP3RIGN       0.705055
PN3RIGRL      0.673202
NP3RIGLL      0.678561
NP3HMOVR      0.754965
NP3HMOVL      0.800974
NP3LGAGR      0.670119
NP3LGAGL      0.730836
NP3RISNG      0.532292
NP3GAIT       0.695233
NP3FRZGT      0.339970
NP3PSTBL      0.594924
NP3POSTR      0.682119
NP3BRADY      0.847141
NP3PTRMR      0.516222
NP3PTRML      0.506818
NP3KTRMR      0.471445
NP3KTRML      0.510347
NP3RTARU      0.713903
NP3RTALU      0.623769
NP3RTARL      0.360840
NP3RTALL      0.330760
NP3RTALJ      0.225927
NP3RTCON      1.085024
DYSKPRES      0.186078
PD_MED_USE    1.424323
dtype: float64

In [13]:
df.columns

Index([u'NP3FACXP', u'NP3SPCH', u'NP3RIGN', u'NP3RIGRU', u'NP3RIGLU', u'PN3RIGRL', u'NP3RIGLL', u'NP3FTAPR', u'NP3FTAPL', u'NP3HMOVR', u'NP3HMOVL', u'NP3PRSPR', u'NP3PRSPL', u'NP3TTAPR', u'NP3TTAPL', u'NP3LGAGR', u'NP3LGAGL', u'NP3RISNG', u'NP3GAIT', u'NP3FRZGT', u'NP3PSTBL', u'NP3POSTR', u'NP3BRADY', u'NP3PTRMR', u'NP3PTRML', u'NP3KTRMR', u'NP3KTRML', u'NP3RTARU', u'NP3RTALU', u'NP3RTARL', u'NP3RTALL', u'NP3RTALJ', u'NP3RTCON', u'DYSKPRES', u'NHY', u'PD_MED_USE', u'DIAG'], dtype='object')